In [8]:
import pandas as pd

In [9]:
file_path = "C:/Users/user/Desktop/han-toss-dashboard/data/ecos.csv"
df = pd.read_csv(file_path)
df

,항목명1,수도권 아파트 매매 가격지수,한국은행 기준금리,실업률,건설비 공사지수,소비자물가지수,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,단위,2021.6=100,연%,%,2015=100,2020=100,NaN,NaN,NaN,NaN,NaN,NaN
1,2015/01,73.1,2,3.7,100.40,94.861,NaN,NaN,NaN,NaN,NaN,NaN
2,2015/02,73.3,2,4.5,100.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015/03,73.7,1.75,4,100.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015/04,74.3,1.75,3.9,99.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
107,2023/11,93.1,3.5,2.3,153.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,2023/12,93,3.5,3.3,153.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,2024/01,92.7,3.5,3.7,154.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,2024/02,92.5,3.5,3.2,154.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
type(df)

pandas.core.frame.DataFrame

In [17]:
df.drop(['Unnamed: 6', 'Unnamed: 7'], axis = 1)
df.head()

,항목명1,수도권 아파트 매매 가격지수,한국은행 기준금리,실업률,건설비 공사지수,소비자물가지수,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,단위,2021.6=100,연%,%,2015=100,2020=100,NaN,NaN,NaN,NaN,NaN,NaN
1,2015/01,73.1,2,3.7,100.40,94.861,NaN,NaN,NaN,NaN,NaN,NaN
2,2015/02,73.3,2,4.5,100.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015/03,73.7,1.75,4,100.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015/04,74.3,1.75,3.9,99.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df = df.loc[:,'항목명1':'소비자물가지수']
df

,항목명1,수도권 아파트 매매 가격지수,한국은행 기준금리,실업률,건설비 공사지수,소비자물가지수
0,단위,2021.6=100,연%,%,2015=100,2020=100
1,2015/01,73.1,2,3.7,100.40,94.861
2,2015/02,73.3,2,4.5,100.18,NaN
3,2015/03,73.7,1.75,4,100.07,NaN
4,2015/04,74.3,1.75,3.9,99.76,NaN
...,...,...,...,...,...,...
107,2023/11,93.1,3.5,2.3,153.31,NaN
108,2023/12,93,3.5,3.3,153.22,NaN
109,2024/01,92.7,3.5,3.7,154.52,NaN
110,2024/02,92.5,3.5,3.2,154.83,NaN


In [40]:
df_test = df.loc[[1,13,25,37,49,61,73,85,97,109],:]
df = pd.DataFrame(df_test)
df

,항목명1,수도권 아파트 매매 가격지수,한국은행 기준금리,실업률,건설비 공사지수,소비자물가지수
1,2015/01,73.1,2,3.7,100.40,94.861
13,2016/01,77.5,1.5,3.7,100.35,95.783
25,2017/01,78.9,1.25,3.7,106.12,97.645
37,2018/01,81.4,1.5,3.7,110.65,99.086
49,2019/01,83.7,1.75,4.5,115.96,99.466
61,2020/01,84.3,1.25,4.1,118.30,100
73,2021/01,92.5,0.5,5.7,124.12,102.5
85,2022/01,107.9,1.25,4.1,141.91,107.72
97,2023/01,94.9,3.5,3.6,150.84,111.59
109,2024/01,92.7,3.5,3.7,154.52,NaN
